In [19]:
import os
import json
import base64
import io
from mistralai import Mistral
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import markdown
from bs4 import BeautifulSoup



# Load API key from .env file (create this file with your API key)
MISTRAL_API_KEY = "6X71LCz0rZckardeNbmor17ONKFO1NVS"  # Replace with your Mistral API key
api_key = MISTRAL_API_KEY
client = Mistral(api_key=api_key)

document_url = "https://github.com/jvankooo/ocrpro/raw/main/Phable%20CAM%20Final.pdf"



In [8]:
ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "document_url",
        "document_url": document_url
    },
    include_image_base64=True
)


In [18]:
import json

# Save the OCR response to a JSON file
ocr_json_path = "ocr_response.json"

with open(ocr_json_path, "w", encoding="utf-8") as f:
    json.dump(ocr_response.dict(), f, indent=4, ensure_ascii=False)

print(f"OCR response saved to {ocr_json_path}")



OCR response saved to ocr_response.json


C:\Users\chira\AppData\Local\Temp\ipykernel_24340\1437944098.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json.dump(ocr_response.dict(), f, indent=4, ensure_ascii=False)


In [ ]:
import json
import os

# Load the OCR response JSON
ocr_json_path = "ocr_response.json"

with open(ocr_json_path, "r", encoding="utf-8") as f:
    ocr_data = json.load(f)

print("OCR response loaded successfully!")


OCR response loaded successfully!


In [24]:
# Create output directory
os.makedirs("outputs", exist_ok=True)

text_file_path = "outputs/text_output.txt"

with open(text_file_path, "w", encoding="utf-8") as f:
    for page in ocr_data["pages"]:
        f.write(f"Page {page['index'] + 1}:\n")
        f.write(page["markdown"] + "\n\n")
        f.write("="*80 + "\n\n")

print(f"Text content saved to {text_file_path}")


Text content saved to outputs/text_output.txt


In [32]:
for page in ocr_data["pages"]:
    markdown_text = page["markdown"]
    if "|" in markdown_text:  # Check if table-like structure exists
        print(f"Page {page['index']} has a table-like structure:")
        print(markdown_text[:1000])  # Print first 1000 characters to inspect
        print("="*80)


Page 0 has a table-like structure:
| Proposal Summary |  | December, 2021 |  |
| :-- | :--: | :--: | :--: |
| Borrower Name: Terrals Technologies Pvt. Ltd. |  |  |  |
| Sector: Chronic Disease Management and Healthcare Services Platform |  |  |  |
| Proposed Amount: Up to INR 45,00,00,000 |  |  |  |
| Tranche 1: INR 10,00,00,000 |  |  |  |
| Tranche 2: Up to 15\% of the equity raise (Series B) with an overall cap of INR 35Cr |  |  |  |
| Tenor: 18 months from the date of disbursement of each tranche |  |  |  |
| Debt XIRR: 16.56\%; Warrants: 7\%; XIRR with warrant upside: 31.2\% |  |  |  |
| Repayment Schedule: |  |  |  |
| Quarterly Amortization (5 instalments); Moratorium for three months i.e., first principal instalment at the end of 6 |  |  |  |
| months. The coupons are to be serviced monthly on the last day of each month. |  |  |  |
| Filtering Criteria: |  |  |  |
| Criteria | Stride Filter | Pre-Equity | Post-Equity* |
| Equity Raised till Date | $>$ INR 50Cr | INR 64Cr | INR 1

In [51]:
import json
import pandas as pd
import re
import os

def extract_tables_from_json(json_file_path):
    # Load the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # List to store all tables
    all_tables = []
    
    # Function to extract tables from markdown text
    def extract_markdown_tables(text, page_num=None):
        # Pattern to match markdown tables
        table_pattern = r'(\|.*\|[\s]*\n\|[\s]*:?-+:?[\s]*\|.*\n(?:\|.*\|[\s]*\n)*)'
        
        # Find all tables in the text
        tables = re.findall(table_pattern, text)
        
        # Add page number information if available
        if page_num is not None:
            return [(table, page_num) for table in tables]
        return [(table, None) for table in tables]
    
    # Check if the JSON has a pages structure
    if isinstance(data, dict) and 'pages' in data:
        for i, page in enumerate(data['pages']):
            if 'markdown' in page:
                tables = extract_markdown_tables(page['markdown'], i)
                all_tables.extend(tables)
    # Alternative structure: list of pages
    elif isinstance(data, list) and data and isinstance(data[0], dict) and 'markdown' in data[0]:
        for i, page in enumerate(data):
            if 'markdown' in page:
                tables = extract_markdown_tables(page['markdown'], i)
                all_tables.extend(tables)
    # If it's not in the expected structure, try to search in any string field
    else:
        def search_in_obj(obj, path=""):
            tables = []
            if isinstance(obj, dict):
                for key, value in obj.items():
                    new_path = f"{path}.{key}" if path else key
                    if key == "markdown" and isinstance(value, str):
                        # Extract page number if possible
                        page_match = re.search(r'page[_\s]*(\d+)', new_path, re.IGNORECASE)
                        page_num = int(page_match.group(1)) if page_match else None
                        tables.extend(extract_markdown_tables(value, page_num))
                    elif isinstance(value, (dict, list)):
                        tables.extend(search_in_obj(value, new_path))
            elif isinstance(obj, list):
                for i, item in enumerate(obj):
                    tables.extend(search_in_obj(item, f"{path}[{i}]"))
            return tables
        
        all_tables = search_in_obj(data)
    
    return all_tables

def clean_cell_content(cell_text):
    # Remove non-printable characters and control characters
    clean_text = ''.join(char for char in cell_text if ord(char) >= 32 or char in '\n\t')
    # Remove excessive whitespace
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

def parse_markdown_table(table_text):
    # Split the table into lines
    lines = table_text.strip().split('\n')
    
    # Find the header row and separator row
    header_row = None
    separator_row = None
    data_rows = []
    
    for i, line in enumerate(lines):
        line = line.strip()
        if not line.startswith('|') or not line.endswith('|'):
            continue
            
        if separator_row is None and (':--' in line or '---' in line):
            separator_row = i
            header_row = i - 1 if i > 0 else None
        elif separator_row is not None:
            data_rows.append(i)
    
    # If we didn't find a separator row, try a simpler approach
    if separator_row is None:
        if len(lines) >= 2:
            header_row = 0
            data_rows = list(range(1, len(lines)))
    
    # Process rows
    processed_rows = []
    
    # Process all rows (including header if present)
    rows_to_process = ([header_row] if header_row is not None else []) + data_rows
    
    for i in rows_to_process:
        if i is None or i >= len(lines):
            continue
            
        line = lines[i].strip()
        if not line.startswith('|') or not line.endswith('|'):
            continue
            
        # Remove leading and trailing pipes and split by pipe
        cells = line.strip('|').split('|')
        # Clean up whitespace and remove non-printable characters in each cell
        cells = [clean_cell_content(cell) for cell in cells]
        processed_rows.append(cells)
    
    # Create DataFrame
    if processed_rows:
        if header_row is not None and len(processed_rows) > 0:
            headers = processed_rows[0]
            data = processed_rows[1:] if len(processed_rows) > 1 else []
        else:
            # Create default headers
            if processed_rows:
                max_cols = max(len(row) for row in processed_rows)
                headers = [f"Column{i+1}" for i in range(max_cols)]
                data = processed_rows
            else:
                return pd.DataFrame()
        
        # Clean header names
        headers = [clean_cell_content(h) if h else f"Column{i+1}" for i, h in enumerate(headers)]
        
        # Ensure all rows have the same number of columns
        max_cols = len(headers)
        for i in range(len(data)):
            if len(data[i]) < max_cols:
                data[i].extend([''] * (max_cols - len(data[i])))
            elif len(data[i]) > max_cols:
                data[i] = data[i][:max_cols]
        
        df = pd.DataFrame(data, columns=headers)
        return df
    
    return pd.DataFrame()

def save_tables_to_excel(tables, output_excel_path):
    # Remove file if it already exists to avoid issues
    if os.path.exists(output_excel_path):
        try:
            os.remove(output_excel_path)
            print(f"Removed existing file: {output_excel_path}")
        except Exception as e:
            print(f"Warning: Could not remove existing file: {e}")
    
    # Create a list to store DataFrames with their sheet names
    dfs_to_save = []
    tables_saved = 0
    
    # First, parse all tables and prepare them
    for i, (table, page_num) in enumerate(tables):
        df = parse_markdown_table(table)
        
        if not df.empty and len(df) > 0:
            # Create a sheet name with page information if available
            sheet_name = f"Page{page_num}_Table{i+1}" if page_num is not None else f"Table_{i+1}"
            
            # Excel sheet names must be <= 31 characters
            if len(sheet_name) > 31:
                sheet_name = sheet_name[:31]
            
            # Store the DataFrame and its sheet name
            dfs_to_save.append((df, sheet_name))
            tables_saved += 1
            print(f"Prepared table {i+1}" + (f" from page {page_num}" if page_num is not None else ""))
    
    # Now save all prepared DataFrames to Excel
    if dfs_to_save:
        try:
            with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
                used_sheet_names = set()
                
                for df, sheet_name in dfs_to_save:
                    # Make sure sheet name is unique
                    original_name = sheet_name
                    counter = 1
                    while sheet_name in used_sheet_names:
                        sheet_name = f"{original_name[:27]}_{counter}"
                        counter += 1
                    
                    used_sheet_names.add(sheet_name)
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    
            print(f"\nSuccessfully saved {tables_saved} tables to {output_excel_path}")
        except Exception as e:
            print(f"Error saving Excel file: {e}")
            
            # Alternative approach - save each table as a separate CSV file
            print("Attempting to save tables as individual CSV files instead...")
            csv_folder = os.path.splitext(output_excel_path)[0] + "_csv"
            os.makedirs(csv_folder, exist_ok=True)
            
            for i, (df, sheet_name) in enumerate(dfs_to_save):
                safe_name = re.sub(r'[^\w\-_]', '_', sheet_name)
                csv_path = os.path.join(csv_folder, f"{safe_name}.csv")
                df.to_csv(csv_path, index=False)
                print(f"Saved table to {csv_path}")
                
            print(f"Saved {len(dfs_to_save)} tables as CSV files in {csv_folder}")
    else:
        print("No tables to save.")
    
    return tables_saved

def main():
    # File paths
    json_file_path = 'ocr_response.json'  # Replace with your JSON file path
    output_excel_path = 'extracted_tables.xlsx'
    
    # Extract tables
    tables = extract_tables_from_json(json_file_path)
    
    # Check if tables were found
    if tables:
        print(f"Found {len(tables)} tables in the JSON file")
        tables_saved = save_tables_to_excel(tables, output_excel_path)
        if tables_saved == 0:
            print("No valid tables could be parsed from the found content")
    else:
        print("No markdown tables found in the JSON file")
        
        # Debug: Let's print some information about the JSON structure
        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
        print("\nJSON structure information for debugging:")
        if isinstance(data, dict):
            print(f"Top-level keys: {', '.join(data.keys())}")
            if 'pages' in data and isinstance(data['pages'], list):
                print(f"Contains {len(data['pages'])} pages")
                if data['pages'] and isinstance(data['pages'][0], dict):
                    print(f"First page keys: {', '.join(data['pages'][0].keys())}")
        elif isinstance(data, list):
            print(f"JSON contains a list with {len(data)} items")
            if data and isinstance(data[0], dict):
                print(f"First item keys: {', '.join(data[0].keys())}")

if __name__ == "__main__":
    main()

Found 36 tables in the JSON file
Removed existing file: extracted_tables.xlsx
Prepared table 1 from page 0
Prepared table 2 from page 1
Prepared table 3 from page 1
Prepared table 4 from page 2
Prepared table 5 from page 4
Prepared table 6 from page 5
Prepared table 7 from page 5
Prepared table 8 from page 6
Prepared table 9 from page 7
Prepared table 10 from page 12
Prepared table 11 from page 13
Prepared table 12 from page 13
Prepared table 13 from page 13
Prepared table 14 from page 14
Prepared table 16 from page 16
Prepared table 17 from page 17
Prepared table 18 from page 17
Prepared table 19 from page 18
Prepared table 21 from page 19
Prepared table 22 from page 19
Prepared table 24 from page 21
Prepared table 25 from page 21
Prepared table 26 from page 28
Prepared table 27 from page 29
Prepared table 28 from page 29
Prepared table 29 from page 30
Prepared table 30 from page 30
Prepared table 31 from page 30
Prepared table 32 from page 31
Prepared table 33 from page 31
Prepared t

In [28]:
import base64
import io
from PIL import Image

# Create output directory for images
os.makedirs("outputs/images", exist_ok=True)

image_count = 0

for page in ocr_data["pages"]:
    if "images" in page:
        for image in page["images"]:
            if "image_base64" in image:
                image_data = base64.b64decode(image["image_base64"].split(",")[1])
                image_path = f"outputs/images/{image['id']}"
                
                # Save the image
                with open(image_path, "wb") as img_file:
                    img_file.write(image_data)

                # Verify the image integrity
                try:
                    with Image.open(image_path) as img:
                        img.verify()  # Ensure it's a valid image
                    #print(f"Saved: {image_path}")
                    image_count += 1
                except Exception as e:
                    print(f"Corrupt Image Skipped: {image['id']} - {e}")

print(f"Extracted {image_count} images. Saved in 'outputs/images/'")


Extracted 29 images. Saved in 'outputs/images/'
